In [ ]:
#from google.colab.output import eval_js
#print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://zov8lnjjzan-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flask_ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 2YUxQ8B4ia0gq9a6IVjcW43hxyR_DnceP4bNSTvrPdUMrS28 #  <-------En caso de no cargar la pagina, ingrese un nuevo authtoken.

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15964 sha256=4444e252bb18db48a99c2d3c7692e6db7cd1975fd57fce9a325180531b731640
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, render_template, request, redirect, url_for, send_from_directory
import os
from werkzeug.utils import secure_filename
import cv2
from flask_ngrok import run_with_ngrok
from keras.models import load_model
import numpy as np


app = Flask(__name__, template_folder="/content/drive/MyDrive/Carpeta_flask/templates", static_folder="/content/drive/MyDrive/Carpeta_flask/static")
run_with_ngrok(app)  # Inicia ngrok cuando la app se ejecute
app.config['UPLOAD_FOLDER'] = '/content/drive/MyDrive/Carpeta_flask/uploads'
app.config['FRAME_FOLDER'] = '/content/drive/MyDrive/Carpeta_flask/frames'
app.config['ALLOWED_EXTENSIONS'] = {'mp4', 'avi', 'mov'}
model = load_model('/content/drive/MyDrive/PDI_CNN_NORMAL/mi_modelo_vgg16.h5')

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

if not os.path.exists(app.config['FRAME_FOLDER']):
    os.makedirs(app.config['FRAME_FOLDER'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

# Cargar el modelo guardado

# Preprocesamiento de la imagen para la predicción
def preprocess_image(img):

    if img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    return img

# Función de predicción
def predict(model, img):
    img_array = preprocess_image(img)
    predictions = model.predict(img_array)
    idx = np.argmax(predictions)
    return idx  # Devuelve directamente el índice de la clase

def extract_frames(video_path, frame_folder):
    if not os.path.exists(frame_folder):
        os.makedirs(frame_folder)

    # Abrir el video
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0

    while success:
        # Guardar el fotograma actual en el directorio
        frame_path = os.path.join(frame_folder, f"frame{count}.jpg")
        cv2.imwrite(frame_path, image)

        # Leer el siguiente fotograma del video
        success, image = vidcap.read()
        count += 1

    vidcap.release()

    # Ordenar los archivos de fotogramas
    frame_files = os.listdir(frame_folder)
    frame_files.sort(key=lambda x: int(x.split('frame')[1].split('.jpg')[0]))
    return frame_files


def clear_frames(frame_folder):
    for file in os.listdir(frame_folder):
        file_path = os.path.join(frame_folder, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)


@app.route('/', methods=['GET', 'POST'])
def upload_video():
    if request.method == 'POST':
        # Verifica si el archivo está presente en la solicitud
        if 'file' not in request.files:
            return redirect(request.url)

        # Obtiene el archivo de la solicitud
        file = request.files['file']

        # Verifica si el nombre del archivo no está vacío y si es un archivo permitido
        if file.filename == '':
            return redirect(request.url)
        if file and allowed_file(file.filename):
            # El nombre del archivo es seguro para guardar en el sistema de archivos
            filename = secure_filename(file.filename)
            video_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(video_path)

            # Extraer fotogramas del video y obtener los nombres de los archivos de los fotogramas
            frame_files = extract_frames(video_path, app.config['FRAME_FOLDER'])

            # Realizar predicciones en cada fotograma
            predictions = []
            for frame_file in frame_files:
                frame_path = os.path.join(app.config['FRAME_FOLDER'], frame_file)
                frame = cv2.imread(frame_path)
                class_index = predict(model, frame)
                predictions.append(class_index)

            # Procesar los resultados de las predicciones
            banana_count = predictions.count(0)
            apple_count = predictions.count(1)

            # Borrar todos los fotogramas de la carpeta frames
            clear_frames(app.config['FRAME_FOLDER'])

            # Enviar al usuario a la página de éxito con los resultados de las predicciones
            return render_template('result.html', banana_count=banana_count, apple_count=apple_count)

    # Mostrar el formulario de carga si no es una solicitud POST o si la carga falla
    return render_template('index2.html')


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://36cb-34-67-160-47.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [21/Nov/2023 21:28:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2023 21:28:24] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2023 21:28:24] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 30ms/step


INFO:werkzeug:127.0.0.1 - - [21/Nov/2023 21:29:19] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2023 21:29:20] "GET /static/style2.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2023 21:44:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2023 21:44:02] "GET /static/styles.css HTTP/1.1" 304 -


1/1 [==============================] - 0s 18ms/step


INFO:werkzeug:127.0.0.1 - - [21/Nov/2023 21:45:28] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2023 21:45:29] "GET /static/style2.css HTTP/1.1" 304 -
